In [1]:
from safetensors.torch import load_file
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
#from peft import LoraConfig, get_peft_model
from sagemaker.pytorch import PyTorch
import numpy as np
import torch.nn as nn
import shutil

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
!pip install sentencepiece
!pip install tiktoken

  Using cached tiktoken-0.9.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tiktoken-0.9.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)


In [3]:
import transformers
import sentencepiece

In [4]:
transformers.__version__

'4.48.3'

In [5]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from safetensors.torch import load_file
import torch
from sagemaker.serve.mode.function_pointers import Mode
from transformers import AutoTokenizer, AutoModelForCausalLM
from sagemaker import get_execution_role, Session, image_uris
from sagemaker.serve import InferenceSpec
import boto3
from sagemaker.serve import CustomPayloadTranslator
import torch
from torchvision.transforms import transforms
from sagemaker.serve.spec.inference_spec import InferenceSpec
from transformers import pipeline
from sagemaker.serve import ModelServer
from sagemaker.pytorch import PyTorchModel
import sagemaker
from pathlib import Path
import numpy as np
import io
import os
sagemaker_session = Session()
region = boto3.Session().region_name

# get execution role
# please use execution role if you are using notebook instance or update the role arn if you are using a different role
execution_role = get_execution_role() if get_execution_role() is not None else "your-role-arn"

2025-02-25 00:23:20.460535: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
from transformers import LlamaForCausalLM

In [7]:
#%%capture
#!pip install -U accelerate

In [8]:
SAVED_MODEL = "Alexis-Az/Story-Generation-LlaMA-3.1-8B-10k"
INFER_MODEL= "Alexis-Az/Story-Generation-Model"
max_seq_length = 1024

# The PEFT Weighted Merged Model 
In this notebook the peft/lora finetuned model that was finetuned on the tiny stories dataset was formatted successfully and unsuccessfully for deployment. The finetuned models weights were merged so that the lora weights were merged with the original weights as the first step prior to inference so that the model was making predictions based on the full knowledge. **Ultimately, the notebooks used for model deployment in Sagemaker are a work-in-progress experiment to test Sagemaker's API in python for model deployment strategy.**

## Formatting Results

The model was successfully formatted to both a weights dictionary file, as well as a torch.jit .pt model file. The torch.script.trace method is used to compile a model for deployment and is recommended by aws/sagemaker. The model was unsuccessfully deployed to an AWS sagemaker container using their ModelBuilder api methods however, due to multiple causes - a mismatch in model loading method using the recommended AWS image used for inference and the model class used. **Instead, to deploy the LLM, the model is deployed using HuggingFace's endpoint GUI by using the non-formatted model repo containing the merged lora weights and tokenizer.**

### Note:
It is not recommended to run this notebook in a AWS Sagemaker domain spaces environment fully. Firstly, sections of this code work given authentication so a user should be logged in to their own domain. Secondly, sections of this code contain errors due to the environment/container requested by the ModelBuilder during deployment do not align with the model loading method used within the "Story-Generation-LlaMA-3.1-8B-10k" model.

## Loading the Finetuned Model
SAVED_MODEL is the lora huggingface repo of the peft model that has been fine-tuned. INFER_MODEL is the huggingface repo of the model that has the lora weights merged with the base weights for inferring. The model was pushed to the INFER_MODEL huggingface repo, and will be formatted for inference here. To have the model be able to be loaded in a HF TGI inference server, the model's flash attention should be turned off by using the cpu instead of the gpu.

In [ ]:
import unsloth

In [ ]:
adapter_model, tokenizer = unsloth.FastLanguageModel.from_pretrained(SAVED_MODEL, load_in_4bit=True)

In [ ]:
adapter_model.save_pretrained_merged("Story-Generation-LlaMA-3.1-8B-10k", tokenizer)

In [ ]:
adapter_model

In [ ]:
pth_model = AutoModelForCausalLM.from_pretrained("Story-Generation-LlaMA-3.1-8B-10k")

In [12]:
tokenizer = AutoTokenizer.from_pretrained("Story-Generation-LlaMA-3.1-8B-10k")

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
pth_model.push_to(
    INFER_MODEL,
    tokenizer=tokenizer,
    safe_serialization=True,
    create_pr=True,
    max_shard_size="3GB",
)

In [49]:
from tokenizers.pre_tokenizers import Whitespace
from transformers import convert_slow_tokenizer

In [56]:
from transformers import LlamaTokenizerFast

In [48]:
tokenizer.push_to_hub(INFER_MODEL)

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Alexis-Az/Story-Generation-Model/commit/9b912e6efbfac87878ea0dd39a26cfb3f7d876f2', commit_message='Upload tokenizer', commit_description='', oid='9b912e6efbfac87878ea0dd39a26cfb3f7d876f2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Alexis-Az/Story-Generation-Model', endpoint='https://huggingface.co', repo_type='model', repo_id='Alexis-Az/Story-Generation-Model'), pr_revision=None, pr_num=None)

### Attempt #1: Scripting the Model as a .pth TorchScript file
**Note: This section will not run unless using a container using large GPU memory and disk size to store a model in GPU memory, and to store the model file on disk.**

In [74]:
tokenizer = AutoTokenizer.from_pretrained(INFER_MODEL)

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained('./test_tokenizer')

In [65]:
class TritonWrapper(nn.Module):
    
    def __init__(self, model_name: str):
        super().__init__()

        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torchscript=True,
            torch_dtype=torch.float16,
            trust_remote_code=True,
            device_map='auto'
            )
        self.model.output_hidden_states = False
                
    def forward(self, input_ids):
        self.model.eval()
        o = self.model(input_ids, output_hidden_states=False)
        return o[0]

In [66]:
model = TritonWrapper(INFER_MODEL)
model.eval()

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

TritonWrapper(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
          (post_attention_layernorm): LlamaRMSNorm((4096,), 

In [67]:
DUMMY_TEXT = 'hi! can you tell me a story about sonic?'

In [ ]:
batch = _tokenizer('hi! can you tell me a story about sonic?', return_tensors="pt")
output = _tokenizer.decode(model.model.generate(batch['input_ids'].cuda(), max_new_tokens=500)[0])

In [80]:
output

'<|begin_of_text|>hi! can you tell me a story about sonic? sonic is a blue hedgehog who can run fast, jump high, and spin fast. he has a friend named amy, who is a pink human girl. they live in a place called green hill. sonic likes to run and play with amy, and he also likes to eat cheese. what is the bad guy\'s name, who always tries to stop sonic and amy from having fun?\nThe story: One sunny day in Green Hill, Sonic and Amy were running and playing together. They were laughing and having a great time. Sonic was spinning fast and Amy was jumping high. They were so happy and carefree.\nSuddenly, they heard a loud and mean laugh. It was the bad guy, Dr. Eggman. He was a tall, green robot with a big, evil grin on his face. He was always trying to stop Sonic and Amy from having fun. He wanted to take over Green Hill and make everyone else sad and scared.\nSonic and Amy looked at each other and knew they had to stop Dr. Eggman. They decided to team up and find a way to defeat him. They r

In [69]:
model.eval()

TritonWrapper(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
          (post_attention_layernorm): LlamaRMSNorm((4096,), 

In [22]:
model_dir = "./serve"

In [ ]:
os.mkdir(model_dir)

In [15]:
assert model.training == False
with torch.no_grad():
    model.model.eval()
    traced_model = torch.jit.trace(model, batch['input_ids'])
    print("traced_model done")
    torch.jit.save(traced_model, model_dir+f"/{INFER_MODEL.split('/')[-1]}.pt")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
/opt/conda/lib/python3.11/site-packages/transformers/models/llama/modeling_llama.py:729: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:
/opt/conda/lib/python3.11/site-packages/torch/jit/_trace.py:1303: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1142756 / 1539072 (74.2%)
Greatest absolute difference: 0.03125 at index (0, 11, 197) (up to 1e-05 allowed)
Greatest relative difference: 670.2 at index (0, 11, 97898) (up to 1e-05 allowed)
  _check_trace(


traced_model done


# Using Sagemaker's 'ModelBuilder' Class to Format the PyTorch Model Artifacts

This is the default code for the AWS SDK from HuggingFace's Deploy Action in their UI

In [11]:
import json
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [ ]:
%%capture
!pip install -U sagemaker

## Attempt #2: Using the Huggingface ModelBuilder object

In [32]:

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'Alexis-Az/Story-Generation-Model',
	'SM_NUM_GPUS': json.dumps(1),
    'SAGEMAKER_TS_RESPONSE_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600',
}



# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="3.0.1"),
	env=hub,
	role=execution_role, 
)


INFO:sagemaker.image_uris:Defaulting to only available Python version: py311
INFO:sagemaker.image_uris:Defaulting to only supported image scope: gpu.


In [33]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.12xlarge",
	container_startup_health_check_timeout=300,
  )

INFO:sagemaker:Creating model with name: huggingface-pytorch-tgi-inference-2025-02-25-01-49-24-225
INFO:sagemaker:Creating endpoint-config with name huggingface-pytorch-tgi-inference-2025-02-25-01-49-24-880
INFO:sagemaker:Creating endpoint with name huggingface-pytorch-tgi-inference-2025-02-25-01-49-24-880


-------------!

In [34]:
# send request
predictor.predict({
	"inputs": "Hi, what can you help me with?",
})

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/huggingface-pytorch-tgi-inference-2025-02-25-01-49-24-880 in account 597161074694 for more information.

## Attempt #3: Using the Huggingface PiplineModelBuilder Class & Repo

In [23]:
model_dir+f"/{INFER_MODEL.split('/')[-1]}.pt"

'./serve/Story-Generation-Model.pt'

In [24]:
from transformers import pipeline

In [25]:
class InferenceSpec(InferenceSpec):
    def invoke(self, input_object: object, model: object):       
        return model(input_object)
        
    def load(self, model_dir: str):
        return pipeline('text-generation', "Alexis-Az/Story-Generation-Model", device_map='auto')

inf_spec = InferenceSpec()

In [26]:
torch.__version__

'2.4.1.post300'

In [9]:
value: str = "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:"
schema = SchemaBuilder(value,
            {"generated_text": "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\\nDaniel: Hello, Girafatron!\\nGirafatron: Hi, Daniel. I was just thinking about how magnificent giraffes are and how they should be worshiped by all.\\nDaniel: You and I think alike, Girafatron. I think all animals should be worshipped! But I guess that could be a bit impractical...\\nGirafatron: That\'s true. But the giraffe is just such an amazing creature and should always be respected!\\nDaniel: Yes! And the way you go on about giraffes, I could tell you really love them.\\nGirafatron: I\'m obsessed with them, and I\'m glad to hear you noticed!\\nDaniel: I\'"})

In [28]:
local_model_dir = str(Path(model_dir).resolve())

In [29]:
package_dir = 'model.tar.gz'

In [10]:
prompt = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the"
response = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the east coast."

sample_input = {
    "inputs": prompt,
    "parameters": {}
}

sample_output = [
    {
        "generated_text": response
    }
]

In [43]:
instance_type = 'ml.c6a.12xlarge'
image = image_uris.retrieve(region=region, framework='huggingface',base_framework_version='pytorch2.0.0', version = '4.28',image_scope='inference', instance_type=instance_type)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py310


In [48]:
image = '763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:2.0.0-transformers4.28.1-cpu-py310-ubuntu20.04'

In [49]:
model_builder = ModelBuilder(mode=Mode.SAGEMAKER_ENDPOINT,model=INFER_MODEL,schema_builder=SchemaBuilder(sample_input, sample_output), model_path='./huggingface', env_vars={"HF_TASK":"text-generation"}, dependencies={"auto":True}, image_uri=image)

In [50]:
_model = model_builder.build()

ModelBuilder: INFO:     Either inference spec or model is provided. ModelBuilder is not handling MLflow model input
ModelBuilder: WARNING:     HuggingFace JumpStart Model ID not detected. Building for HuggingFace Model ID.
ModelBuilder: WARNING:     HuggingFace JumpStart Model ID not detected. Building for HuggingFace Model ID.
ModelBuilder: WARNING:     HuggingFace Model ID support on model server: None is not currently supported. Defaulting to TGI
ModelBuilder: WARNING:     CUDA is not enabled on your device. [Errno 2] No such file or directory: 'nvidia-smi'. Please run ModelBuilder on CUDA enabled hardware to deploy locally.
ModelBuilder: WARNING:     65.40047919282824 percent of disk space used. Please consider freeing up disk space or increasing the EBS volume if you are on a SageMaker Notebook.
ModelBuilder: WARNING:     Unable to check docker volume utilization at the expected path /var/lib/docker. [Errno 2] No such file or directory: '/var/lib/docker'
ModelBuilder: WARNING:    

In [ ]:
_model.deploy(role=execution_role, instance_type='ml.g4dn.8xlarge')

## Attempt 4: Using the ModelBuilder class and Compiling own Container (BYOC)
The last attempt is the most tedious and less automated method which requires saving the model artifacts and using the sagemaker GUI to create the deployable model/the endpoint. **Note: deploy() in this section results in deployment errors potentially due to the model class used being a CAUSAL_LM model that AWS tries to load using a concrete FlashCausalLM class instead of the LLamaCausalLM class, or due to issues loading the lora weight merged model that has been finetuned using Unsloth. This section will not run unless using a container using large CPU memory.**

In [35]:
model_builder = ModelBuilder(
    mode=Mode.SAGEMAKER_ENDPOINT,
    model_path=model_dir,
    inference_spec=inf_spec,
    role_arn=execution_role,
    image_uri=image,
    schema_builder=schema,
    env_vars={"HF_TASK":"text-generation"},
    model_server=ModelServer.TORCHSERVE
)

In [23]:
_model = model_builder.build()

ModelBuilder: INFO:     Either inference spec or model is provided. ModelBuilder is not handling MLflow model input
ModelBuilder: WARNING:     HuggingFace JumpStart Model ID not detected. Building for HuggingFace Model ID.
ModelBuilder: INFO:     Local instance_type ml.g5.12xlarge detected. ml.g5.12xlarge will be default when deploying to a SageMaker Endpoint. This default can be overriden in model.deploy()
ModelBuilder: WARNING:     65.24585339053998 percent of disk space used. Please consider freeing up disk space or increasing the EBS volume if you are on a SageMaker Notebook.
ModelBuilder: WARNING:     Unable to check docker volume utilization at the expected path /var/lib/docker. [Errno 2] No such file or directory: '/var/lib/docker'


In [ ]:
model.deploy()

In [18]:
!sudo apt install pigz

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pigz is already the newest version (2.6-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [33]:
model_dir

'./serve'

In [37]:
os.chdir(model_dir)

In [39]:
import subprocess

def create_tar_pigz(tar_filename, source_dir):
    # = f"tar -I pigz -cf {tar_filename} {source_dir}"
    #subprocess.run(command, shell=True, check=True)
    command = f"tar -I pigz -cf {tar_filename} {source_dir}"
    subprocess.run(command, shell=True, check=True) 


# packaging the model file itself as the top level 'directory'
create_tar_pigz('../'+package_dir, '.')

In [41]:
os.chdir('..')

In [42]:
import boto3

# Create an S3 client
s3 = boto3.client('s3', region_name=region)  # Replace 'YOUR_REGION' with your bucket's region

bucket_name = 'unsloth-finetuned'      # Name of your S3 bucket

try:
    # Upload the file
    s3.upload_file(package_dir, bucket_name, package_dir)
except Exception as e:
    print(f"Error uploading file: {e}")
